In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd drive/MyDrive/cagliari_extraction

/content/drive/MyDrive/cagliari_extraction


In [4]:
import torch
import torch.nn as nn
import torchvision
from simple_attack import simple_attack
import torchvision.transforms as transforms

In [5]:
victim_model = torchvision.models.mobilenet_v3_small(weights = 'MobileNet_V3_Small_Weights.DEFAULT')

Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


  0%|          | 0.00/9.83M [00:00<?, ?B/s]

In [6]:
substitute_model = torchvision.models.mobilenet_v3_small(weights = None)

In [7]:
trial_attack = simple_attack(substitute_model)

In [8]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 4
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
optimizer = torch.optim.Adam(substitute_model.parameters())
scheduler = None
loss_fn = nn.CrossEntropyLoss()
loss_fn.requires_grad = False
train_config = {'num_epochs': 1, 'optimizer': optimizer, 'scheduler': scheduler, 'loss_fn': loss_fn, 'batch_size': 4}

In [34]:
trial_attack.run(victim_model, 10, train_config, torch.permute(torch.tensor(testset.data).float(), (0, 3, 1, 2)))

number of queries: 100%|██████████| 10/10 [00:00<00:00, 100.28it/s]

0
1
2
3
4
5
6
7
8
9


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 